### Can we classify each phase as relatively low or high anxiety for each subject? ###

In [78]:
# IMPORTING MODULES
import glob
import importlib
import matplotlib.pyplot as plt
import numpy as np
import os
cvx_path = os.path.abspath(os.path.join('..', '..', 'cvxEDA', 'src'))
module_path = os.path.abspath(os.path.join('..', '..', 'src'))
import pandas as pd
import random
import scipy.signal as ss
import shap
import sys
sys.path.append(module_path)

import tools.data_reader_wesad as dr
import tools.display_tools as dt
import tools.preprocessing as preprocessing
import train

import lightgbm as lgb
from lightgbm import LGBMClassifier
from scipy.fft import fft, fftfreq, fftshift
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, \
    mean_absolute_error, mean_squared_error, log_loss
from sklearn.model_selection import train_test_split, cross_val_score, RepeatedKFold
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import normalize
from xgboost import XGBClassifier

import cvxopt.solvers
cvxopt.solvers.options['show_progress'] = False

import warnings
warnings.filterwarnings(
    "ignore", 
    category=RuntimeWarning
)


phases = dr.Phases.PHASE_ORDER


In [79]:
importlib.reload(train)

models = {
    "LGB": LGBMClassifier(),
    "RF": RandomForestClassifier(random_state=16),
    "XGB": XGBClassifier(random_state=16),
    # "random": None
}

parameters = {
    "LGB": [{
        "objective": ["binary"],
        "num_leaves": [10, 20, 30, 40, 50],
        "max_depth": [3, 4, 5, 6, 7],
        "metric": ["mean_absolute_error", "mean_squared_error", "binary_logloss"]
    }],
    "RF": [{
        "n_estimators": [10, 20, 30, 40, 50],
        "max_features": ["sqrt", "0.4"],
        "min_samples_split": [3, 4, 5, 6, 7],
        "random_state": [16]
    }],
    "XGB": [{
        "objective": ["binary:logistic"],
        "learning_rate": [0.01, 0.1, 0.3, 0.5],
        "max_depth": [4, 5, 6, 7],
        "n_estimators": [10, 20, 30, 40, 50],
        "eval_metric": ["error"],
        "use_label_encoder": [False],
        "random_state": [16]
    }],
    # "random": None
}

metrics = [
    train.Metrics.BPM, 
    train.Metrics.RMSSD, 
    train.Metrics.HF_RR, 
    train.Metrics.LF_RR, 
    train.Metrics.SDNN, 
    # train.Metrics.MEAN_SCL, 
    # train.Metrics.SCR_RATE, 
]
# ] + train.Metrics.STATISTICAL

In [83]:
# K-FOLD CROSS-VALIDATION FOR HYPERPARAMETER SELECTION
importlib.reload(train)
importlib.reload(dr)
importlib.reload(dt)
importlib.reload(preprocessing)

model_phases = dr.Phases.PHASE_ORDER

label_type = "stai"
threshold = "fixed"

x, y = train.Train_WESAD.get_wesad_data(
    metrics, model_phases, verbose=False, label_type=label_type, threshold=threshold, 
    normalize=True, combine_phases=False
)
x = x.drop(["phaseId"], axis=1)
inds = pd.isnull(x).any(axis=1).to_numpy().nonzero()[0]
x = x.drop(labels=inds, axis=0).reset_index(drop=True)
y = y.drop(labels=inds, axis=0).reset_index(drop=True)

acc_results = {
    "LGB": [],
    "RF": [],
    "XGB": [],
    # "random": []
}
reports = {
    "LGB": [],
    "RF": [],
    "XGB": [],
    # "random": []
}
best_models = {}

num_iters = 5
get_importance = True
for _ in range(num_iters):
    # HYPERPARAMETER TUNING
    model_data = train.grid_search_cv(
        models, parameters, x, y, by_subject=True, save_metrics=True, is_resample=True,
        get_importance=get_importance, drop_subject=True, test_size=0.1, folds=5
    )

    for model_name in models.keys():
        best_models[model_name] = model_data[model_name]["best_model"]
        print(f"{model_name}: {model_data[model_name]['best_params']}")

    # FEATURE SELECTION
    x_train, y_train = model_data["train"]
    features = {name: metrics for name in models.keys()}
    # features = train.feature_selection(best_models, model_data["cv"], x_train, y_train, n_features=5)

    # TEST USING OPTIMIZED MODELS AND FEATAURES
    x_test, y_test = model_data["test"]
    out = train.train_test_model(best_models, features, x_train, y_train, x_test, y_test)

    for model_name in acc_results:
        acc_results[model_name].append(out[model_name]["performance"][0])
        reports[model_name].append(out[model_name]["performance"][1])
        if get_importance:
            try:
                print("")
                feature_imp = list(zip(metrics + ["lf_hf_ratio"], out[model_name]["performance"][2]))
                feature_imp = sorted(feature_imp, key=lambda x: x[1], reverse=True)
                print(feature_imp)
            except Exception as e:
                print(out[model_name]["performance"][2])
            print("")

for model_name in acc_results.keys():
    print(f"Model evaluation metrics for {model_name}:")
    for i in range(len(reports[model_name])):
        report = reports[model_name][i]
        acc = acc_results[model_name][i]
        p = report["precision"]
        r = report["recall"]
        f1 = report["f1"]
        auc = report["auc"]
        print(f"\tAccuracy: {acc}\n\tPrecision: {p}\n\tRecall: {r}\n\tF1-score: {f1}\n\tAUC score: {auc}\n" + "-"*40)
    print(f"Mean acc: {np.mean([acc_results[model_name][i] for i in range(len(reports[model_name]))])}")
    print(f"Mean F1-score: {np.mean([reports[model_name][i]['f1'] for i in range(len(reports[model_name]))])}")
    print(f"Mean AUC score: {np.mean([reports[model_name][i]['auc'] for i in range(len(reports[model_name]))])}")
    print("\n")

Grid search for SVM ...
Grid search for LGB ...
Grid search for RF ...


One or more of the test scores are non-finite: [0.84899057 0.86216887 0.87141205 0.87798578 0.88432573 0.83787002
 0.86526046 0.85929291 0.87383093 0.87521117 0.84796677 0.87268795
 0.86745498 0.87338313 0.87303327 0.85866278 0.86533767 0.8667493
 0.88166248 0.87867084 0.86193242 0.86973062 0.86773622 0.87177226
 0.87340138        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan]


Grid search for XGB ...


One or more of the test scores are non-finite: [0.8500515  0.84945822 0.85323895 0.85375612 0.85551174 0.85155415
 0.85058661 0.85480009 0.85300906 0.84557218 0.85464404 0.85409784
 0.85458942 0.85343201 0.84739962 0.85379815 0.85318173 0.85304128
 0.85216477 0.84585148 0.87688845 0.89292085 0.9025474  0.90156231
 0.91168122 0.87480382 0.88692698 0.89721848 0.89268466 0.9012632
 0.87848412 0.88512912 0.90411485 0.90221945 0.90500922 0.87469701
 0.88605399 0.89930135 0.89292735 0.89824593 0.90302469 0.91238199
 0.91877907 0.92173448 0.92641987 0.90697507 0.91161139 0.91798065
 0.92248683 0.92538838 0.89578694 0.90850863 0.90907288 0.92060136
 0.92226078 0.90352124 0.90984689 0.91261604 0.92257882 0.92367326
 0.90613922 0.9098498  0.91368999 0.92180833 0.92264066 0.90600617
 0.90622061 0.91428169 0.91351636 0.91531585 0.89986586 0.9179251
 0.92243876 0.92395866 0.9223754  0.89937987 0.91523741 0.92157851
 0.92464974 0.92335855        nan        nan        nan        nan
        nan      

SVM: {'C': 1, 'degree': 6, 'gamma': 1, 'kernel': 'poly'}
LGB: {'max_depth': 4, 'metric': 'mean_absolute_error', 'num_leaves': 20, 'objective': 'binary'}
RF: {'max_features': 'sqrt', 'min_samples_split': 3, 'n_estimators': 50, 'random_state': 16}
XGB: {'eval_metric': 'error', 'learning_rate': 0.3, 'max_depth': 4, 'n_estimators': 50, 'objective': 'binary:logistic', 'random_state': 16, 'use_label_encoder': False}
Training SVM ...
Model SVM, Actual: [0 1], [54 16], Predictions: [0 1], [66  4]
coef_ only available for SVC with linear kernel
Training LGB ...
Model LGB, Actual: [0 1], [54 16], Predictions: [0 1], [63  7]
Training RF ...
Model RF, Actual: [0 1], [54 16], Predictions: [0 1], [63  7]
Training XGB ...
Model XGB, Actual: [0 1], [54 16], Predictions: [0 1], [63  7]

[('bpm', 232), ('mean_SCL', 143), ('hf_rr', 108), ('rmssd', 107), ('sdnn', 71), ('SCR_rate', 69), ('lf_rr', 63)]


[('bpm', 0.37849528518343795), ('rmssd', 0.1912305403502773), ('SCR_rate', 0.10615843959870051), ('sdnn'

...

One or more of the test scores are non-finite: [0.87818628 0.88118661 0.88816648 0.88801503 0.88654032 0.877978
 0.88889342 0.8911009  0.89107208 0.88253092 0.87375068 0.88787002
 0.88855399 0.88007621 0.88266253 0.87685097 0.88918788 0.88795057
 0.87829539 0.88123992 0.88978515 0.89860384 0.89303026 0.89438648
 0.8903992  0.89106594 0.88681355 0.88618545 0.87916696 0.8767813
 0.89115011 0.88647928 0.88450553 0.88339148 0.88076476 0.84419809
 0.88949108 0.88091821 0.88666624 0.89001548 0.88856074 0.88069842
 0.88254125 0.87654687 0.8784584  0.87707149 0.87024031 0.88088096
 0.88157926 0.8806113  0.87225338 0.86365072 0.8783346  0.87951569
 0.878577   0.87577144 0.87712465 0.8841208  0.88560165 0.8880523
 0.86890441 0.86462957 0.86864306 0.87607828 0.87708934 0.87239827
 0.88232807 0.88397959 0.88904052 0.89079832 0.86358265 0.874059
 0.8830071  0.88507538 0.89225338 0.86388997 0.87856101 0.88256062
 0.88681426 0.89073433        nan        nan        nan        nan
        nan        na

SVM: {'C': 10, 'degree': 5, 'gamma': 1, 'kernel': 'poly'}
LGB: {'max_depth': 3, 'metric': 'mean_absolute_error', 'num_leaves': 10, 'objective': 'binary'}
RF: {'max_features': 'sqrt', 'min_samples_split': 6, 'n_estimators': 10, 'random_state': 16}
XGB: {'eval_metric': 'error', 'learning_rate': 0.1, 'max_depth': 4, 'n_estimators': 20, 'objective': 'binary:logistic', 'random_state': 16, 'use_label_encoder': False}
Training SVM ...
Model SVM, Actual: [0 1], [54 16], Predictions: [0 1], [59 11]
coef_ only available for SVC with linear kernel
Training LGB ...
Model LGB, Actual: [0 1], [54 16], Predictions: [0 1], [44 26]
Training RF ...
Model RF, Actual: [0 1], [54 16], Predictions: [0 1], [47 23]
Training XGB ...
Model XGB, Actual: [0 1], [54 16], Predictions: [0 1], [45 25]

[('bpm', 152), ('mean_SCL', 120), ('rmssd', 84), ('sdnn', 54), ('SCR_rate', 51), ('hf_rr', 44), ('lf_rr', 30)]


[('bpm', 0.34631398424237914), ('rmssd', 0.31070160361629295), ('sdnn', 0.11518723161982458), ('mean_SCL'

One or more of the test scores are non-finite: [0.8253668  0.8385965  0.83692172 0.84398831 0.8459223  0.85598729
 0.85983102 0.85702824 0.85968587 0.86221321 0.85690388 0.85899961
 0.85296741 0.85344113 0.85436885 0.86861632 0.86593009 0.85459052
 0.85592054 0.8561302  0.86083887 0.85245269 0.8482057  0.84841297
 0.85101053        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan]


Grid search for XGB ...
SVM: {'C': 1, 'degree': 2, 'gamma': 10, 'kernel': 'poly'}
LGB: {'max_depth': 4, 'metric': 'mean_absolute_error', 'num_leaves': 20, 'objective': 'binary'}
RF: {'max_features': 'sqrt', 'min_samples_split': 6, 'n_estimators': 10, 'random_state': 16}
XGB: {'eval_metric': 'error', 'learning_rate': 0.1, 'max_depth': 4, 'n_estimators': 40, 'objective': 'binary:logistic', 'random_state': 16, 'use_label_encoder': False}
Training SVM ...
Model SVM, Actual: [0 1], [54 16], Predictions: [0 1], [54 16]
coef_ only available for SVC with linear kernel
Training LGB ...
Model LGB, Actual: [0 1], [54 16], Predictions: [0 1], [54 16]
Training RF ...
Model RF, Actual: [0 1], [54 16], Predictions: [0 1], [54 16]
Training XGB ...
Model XGB, Actual: [0 1], [54 16], Predictions: [0 1], [54 16]

[('bpm', 198), ('mean_SCL', 127), ('rmssd', 100), ('sdnn', 99), ('SCR_rate', 75), ('hf_rr', 72), ('lf_rr', 63)]


[('bpm', 0.33866677823168145), ('rmssd', 0.17210373393907344), ('SCR_rate', 0.13

One or more of the test scores are non-finite: [0.84112057 0.84666434 0.85166067 0.87340207 0.87372778 0.84895548
 0.85575823 0.86108893 0.86182658 0.86446303 0.8537774  0.85603115
 0.86142687 0.8616045  0.84653521 0.85295031 0.85431195 0.86026166
 0.86108432 0.84427501 0.85361465 0.85841288 0.89014769 0.90229431
 0.89818043 0.83987123 0.84980133 0.86656619 0.87797773 0.87765053
 0.83000809 0.84733416 0.85623338 0.87172572 0.87621529 0.83654968
 0.84208755 0.85423948 0.87219226 0.87670911 0.85162    0.88481574
 0.88235566 0.87637305 0.87560737 0.84654456 0.8637229  0.86191895
 0.8633608  0.86427598 0.86416953 0.87404518 0.87046411 0.86791753
 0.86635577 0.8445518  0.85032422 0.85582706 0.86012908 0.8639535
 0.88196296 0.87264639 0.87375744 0.87437468 0.8725642  0.87211986
 0.8693091  0.86562176 0.8678985  0.86785841 0.86592051 0.86187384
 0.86464906 0.86822036 0.87076201 0.88559025 0.86257242 0.86475456
 0.86889209 0.86983948        nan        nan        nan        nan
        nan     